**Group 26**

**Names**: student number

**Pedro Cerejeira**: 20230442<br>
**Diogo Reis**: 20230481<br>
**Marta Jesus**: 20230464

**1. Imports:**

In [105]:
# Import Packages
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import warnings
from sklearn.exceptions import DataConversionWarning

# Ignore DataConversion warnings for LinearRegression later on
warnings.filterwarnings(action='ignore', category=UserWarning, module='sklearn')

In [106]:
# Read/Import dataset
data = pd.read_csv('XYZ_sports_dataset.csv', delimiter = ';')
data.head(10)

,ID,Age,Gender,Income,EnrollmentStart,EnrollmentFinish,LastPeriodStart,LastPeriodFinish,DateLastVisit,DaysWithoutFrequency,...,OtherActivities,NumberOfFrequencies,AttendedClasses,AllowedWeeklyVisitsBySLA,AllowedNumberOfVisitsBySLA,RealNumberOfVisits,NumberOfRenewals,HasReferences,NumberOfReferences,Dropout
0,10000,60,Female,5500.0,2019-09-03,2019-10-31,2019-07-01,2019-12-31,2019-10-30,1,...,0.0,9.0,7,NaN,6.28,2,0,0.0,0,0
1,10001,29,Female,2630.0,2014-08-12,2015-09-14,2015-01-01,2015-12-31,2015-07-16,60,...,0.0,23.0,1,2.0,17.42,1,2,0.0,0,1
2,10002,23,Male,1980.0,2017-05-02,2017-06-01,2017-01-01,2017-06-30,2017-05-25,7,...,0.0,6.0,0,7.0,30.03,6,0,0.0,0,1
3,10003,9,Male,0.0,2018-09-05,2019-02-12,2018-07-01,2019-06-30,2019-01-21,22,...,0.0,20.0,2,2.0,17.72,3,0,0.0,0,1
4,10004,35,Male,4320.0,2016-04-20,2018-06-07,2018-01-01,2018-06-30,2017-11-09,210,...,NaN,41.0,0,7.0,60.97,0,3,0.0,0,1
5,10005,24,Female,2220.0,2015-12-01,2016-07-31,2016-01-01,2016-12-31,2016-07-28,3,...,0.0,28.0,0,7.0,59.01,26,0,0.0,0,1
6,10006,23,Male,2340.0,2015-10-06,2019-04-11,2019-01-01,2019-06-30,2019-03-26,16,...,0.0,22.0,0,7.0,30.03,3,3,0.0,0,1
7,10007,23,Male,1910.0,2015-03-25,2019-07-31,2019-01-01,2019-06-30,2019-04-23,99,...,0.0,34.0,0,7.0,60.97,11,5,0.0,0,1
8,10008,29,Male,3220.0,2016-04-08,2017-06-07,2017-01-01,2017-06-30,2017-03-30,69,...,0.0,6.0,0,7.0,60.97,0,2,0.0,0,1
9,10009,14,Female,0.0,2016-07-07,2017-02-01,2016-07-01,2017-06-30,2016-11-19,74,...,0.0,6.0,6,2.0,17.72,0,2,0.0,0,1


**2. Exploratory Analysis:** 

**2.1 Checking Types:** 

In [139]:
# Exploring Types
data.dtypes

ID                              int64
Age                             int64
Gender                         object
Income                        float64
EnrollmentStart                object
EnrollmentFinish               object
LastPeriodStart                object
LastPeriodFinish               object
DateLastVisit                  object
DaysWithoutFrequency            int64
LifetimeValue                 float64
UseByTime                       int64
AthleticsActivities           float64
WaterActivities               float64
FitnessActivities             float64
DanceActivities               float64
TeamActivities                float64
RacketActivities              float64
CombatActivities              float64
NatureActivities              float64
SpecialActivities             float64
OtherActivities               float64
NumberOfFrequencies           float64
AttendedClasses                 int64
AllowedWeeklyVisitsBySLA      float64
AllowedNumberOfVisitsBySLA    float64
RealNumberOf

Here we can conclude that some columns' types must be changed:

note: for the column **Income**, we are keeping the float type, even though it could be converted to integer


In [140]:
# Change the data types of the columns
columns_to_change = {
    'UseByTime': 'bool',
    'AthleticsActivities': 'bool',
    'WaterActivities': 'bool',
    'FitnessActivities': 'bool',
    'DanceActivities': 'bool',
    'TeamActivities': 'bool',
    'RacketActivities': 'bool',
    'CombatActivities': 'bool',
    'NatureActivities': 'bool',
    'SpecialActivities': 'bool',
    'OtherActivities': 'bool',
    'NumberOfFrequencies': 'Int64',
    'AllowedWeeklyVisitsBySLA': 'Int64',
    'HasReferences': 'bool',
    'Dropout': 'bool'
}
df = data.copy()
df = df.astype(columns_to_change)

In [141]:
# Confirm the new column types
df.dtypes

ID                              int64
Age                             int64
Gender                         object
Income                        float64
EnrollmentStart                object
EnrollmentFinish               object
LastPeriodStart                object
LastPeriodFinish               object
DateLastVisit                  object
DaysWithoutFrequency            int64
LifetimeValue                 float64
UseByTime                        bool
AthleticsActivities              bool
WaterActivities                  bool
FitnessActivities                bool
DanceActivities                  bool
TeamActivities                   bool
RacketActivities                 bool
CombatActivities                 bool
NatureActivities                 bool
SpecialActivities                bool
OtherActivities                  bool
NumberOfFrequencies             Int64
AttendedClasses                 int64
AllowedWeeklyVisitsBySLA        Int64
AllowedNumberOfVisitsBySLA    float64
RealNumberOf

**2.2 Checking nans:** 

In [142]:
# count of missing values
df.isna().sum()

ID                              0
Age                             0
Gender                          0
Income                        495
EnrollmentStart                 0
EnrollmentFinish                0
LastPeriodStart                 0
LastPeriodFinish                0
DateLastVisit                   0
DaysWithoutFrequency            0
LifetimeValue                   0
UseByTime                       0
AthleticsActivities             0
WaterActivities                 0
FitnessActivities               0
DanceActivities                 0
TeamActivities                  0
RacketActivities                0
CombatActivities                0
NatureActivities                0
SpecialActivities               0
OtherActivities                 0
NumberOfFrequencies            26
AttendedClasses                 0
AllowedWeeklyVisitsBySLA      535
AllowedNumberOfVisitsBySLA      0
RealNumberOfVisits              0
NumberOfRenewals                0
HasReferences                   0
NumberOfRefere

**Income has a lot of nans, lets analyze them:**

Income and Age should have a high correlation, lets check.

In [143]:
# Finding the correlation between Income and Age

df['Income'].corr(df['Age'])

0.8753525249372762

Okay, 0.87 is a very high correlation, lets use Age in order to estimate the 'nans' in Income<br>
Lets try to find some outliers.

Under aged individuals should have no income or low income, lets try to prove this

In [144]:
# Filter the DataFrame to select individuals under 18 with non-zero income
under_18_with_income = df[(df['Age'] <= 18) & (df['Income'] > 0)]
print(len(under_18_with_income))

429


Okay actually a good amount of underaged individuals has an income, lets check the % of kids with an income for each age

In [145]:
# Group the data by 'Age' (of kids with income) and count them. How many kids have income per age
age_groups = under_18_with_income.groupby('Age').size()

# How many individuals exist for each age (total)
total_counts = df[df['Age'] <= 18].groupby('Age').size()

# Calculate the percentage for each age group
percentage_by_age = (age_groups / total_counts * 100).fillna(0)

# Create a DataFrame to store the results
result_df = pd.DataFrame({'Percentage': percentage_by_age})
result_df.index.name = 'Age'

# Print or display the result
print(result_df)

     Percentage
Age            
0      0.000000
1      0.000000
2      0.938967
3      0.000000
4      0.574713
5      1.111111
6      0.000000
7      0.000000
8      0.000000
9      0.675676
10     0.000000
11     0.793651
12     0.000000
13     1.694915
14     1.492537
15     5.325444
16     9.278351
17   100.000000
18    99.565217


If the percentage is **less than 2%**, we are assuming them as **outliers**. Lets drop them

In [146]:
# Under 2% are all the individuals under 15 years old which have an income
df = df[~((df['Age'] < 15) & (df['Income'] > 0))]

How many of those kids have nan in Income?

In [147]:
len(df[(df['Age'] < 15) & df['Income'].isna()])

313

Following the same logic, lets also fill nans of individuals under 15 years old with 0

In [148]:
df.loc[(df['Age'] < 15) & df['Income'].isna(), 'Income'] = 0

Lets now check the correlation once again

In [149]:
# Finding the new correlation between Income and Age

df['Income'].corr(df['Age'])

0.8808106043038966

Okay the correlation is better, 0.88 now, lets see how many nans are still in Income, we only handled under 15 years old

In [150]:
# count of missing values
df.isna().sum()

ID                              0
Age                             0
Gender                          0
Income                        182
EnrollmentStart                 0
EnrollmentFinish                0
LastPeriodStart                 0
LastPeriodFinish                0
DateLastVisit                   0
DaysWithoutFrequency            0
LifetimeValue                   0
UseByTime                       0
AthleticsActivities             0
WaterActivities                 0
FitnessActivities               0
DanceActivities                 0
TeamActivities                  0
RacketActivities                0
CombatActivities                0
NatureActivities                0
SpecialActivities               0
OtherActivities                 0
NumberOfFrequencies            26
AttendedClasses                 0
AllowedWeeklyVisitsBySLA      535
AllowedNumberOfVisitsBySLA      0
RealNumberOfVisits              0
NumberOfRenewals                0
HasReferences                   0
NumberOfRefere

Lets assume that **individuals above 15 years old can receive income**, how can we fill their nans approximatly?

We could use the mean for each age, although, given the high correlation, a linear regression model can provide a more precise estimation for the missing income values based on age. It can capture the linear trend between the two variables more effectively.

In [151]:
# Filter the data for individuals with valid 'Age' and 'Income' values, LinearRegression requires no nans
regression_data = df.dropna(subset=['Age', 'Income'])

# Filter further for individuals older than 15
regression_data = regression_data[regression_data['Age'] >= 15]

X = regression_data[['Age']]
y = regression_data['Income']

model = LinearRegression()
model.fit(X, y)

# Create a range of ages from the minimum to the maximum age in the dataset (15+ years old)
ages_to_estimate = range(regression_data['Age'].min(), regression_data['Age'].max() + 1)

estimated_incomes = model.predict(np.array(ages_to_estimate).reshape(-1, 1))

income_estimates_df = pd.DataFrame({
    'Age': ages_to_estimate,
    'Estimated_Income': estimated_incomes
})

print(income_estimates_df)

    Age  Estimated_Income
0    15       1225.690979
1    16       1318.620418
2    17       1411.549857
3    18       1504.479296
4    19       1597.408735
..  ...               ...
68   83       7544.892825
69   84       7637.822264
70   85       7730.751703
71   86       7823.681142
72   87       7916.610581

[73 rows x 2 columns]


Lets now fill the nan values with this estimatives

In [152]:
# Lets use our income estimates to fill the nan values. First we need this as a dict
age_to_income_map = income_estimates_df.set_index('Age')['Estimated_Income'].to_dict()

# Now, fill NaN values in the 'Income' column based on the 'Age' column
df['Income'] = df['Income'].fillna(df['Age'].map(age_to_income_map))

In [153]:
# count of missing values
df.isna().sum()

ID                              0
Age                             0
Gender                          0
Income                          0
EnrollmentStart                 0
EnrollmentFinish                0
LastPeriodStart                 0
LastPeriodFinish                0
DateLastVisit                   0
DaysWithoutFrequency            0
LifetimeValue                   0
UseByTime                       0
AthleticsActivities             0
WaterActivities                 0
FitnessActivities               0
DanceActivities                 0
TeamActivities                  0
RacketActivities                0
CombatActivities                0
NatureActivities                0
SpecialActivities               0
OtherActivities                 0
NumberOfFrequencies            26
AttendedClasses                 0
AllowedWeeklyVisitsBySLA      535
AllowedNumberOfVisitsBySLA      0
RealNumberOfVisits              0
NumberOfRenewals                0
HasReferences                   0
NumberOfRefere

Nice, Income is solved, lets handle NumberOfFrequencies now

For this column, since the number of nan values is low, lets use the median to better fill null values (median is more resistant to outliers)

In [154]:
df["NumberOfFrequencies"].fillna(round(df["NumberOfFrequencies"].median(), 0), inplace=True)

In [155]:
df.isna().sum()

ID                              0
Age                             0
Gender                          0
Income                          0
EnrollmentStart                 0
EnrollmentFinish                0
LastPeriodStart                 0
LastPeriodFinish                0
DateLastVisit                   0
DaysWithoutFrequency            0
LifetimeValue                   0
UseByTime                       0
AthleticsActivities             0
WaterActivities                 0
FitnessActivities               0
DanceActivities                 0
TeamActivities                  0
RacketActivities                0
CombatActivities                0
NatureActivities                0
SpecialActivities               0
OtherActivities                 0
NumberOfFrequencies             0
AttendedClasses                 0
AllowedWeeklyVisitsBySLA      535
AllowedNumberOfVisitsBySLA      0
RealNumberOfVisits              0
NumberOfRenewals                0
HasReferences                   0
NumberOfRefere

Finally lets handle the column AllowedWeeklyVisitsBySLA

In [157]:
# Lets see the correlations of all features with the 'AllowedWeeklyVisitsBySLA'
correlation = df.corr(numeric_only = True)
print(correlation['AllowedWeeklyVisitsBySLA'])

ID                           -0.009779
Age                           0.324157
Income                        0.339814
DaysWithoutFrequency         -0.011010
LifetimeValue                -0.313766
UseByTime                     0.111082
AthleticsActivities           0.014495
WaterActivities              -0.508415
FitnessActivities             0.589821
DanceActivities              -0.006433
TeamActivities               -0.221603
RacketActivities             -0.238414
CombatActivities              0.140488
NatureActivities              0.003655
SpecialActivities            -0.004299
OtherActivities              -0.040739
NumberOfFrequencies           0.018813
AttendedClasses              -0.457501
AllowedWeeklyVisitsBySLA      1.000000
AllowedNumberOfVisitsBySLA    0.674454
RealNumberOfVisits            0.161452
NumberOfRenewals             -0.186531
HasReferences                -0.170145
NumberOfReferences           -0.166825
Dropout                       0.183685
Name: AllowedWeeklyVisits

Since 'AllowedWeeklyVisitsBySLA' doesn't have a high correlation with another feature, instead of using the linear regression, we will fill the NANs with the median like we did for the 'NumberOfFrequencies' column

In [158]:
df["AllowedWeeklyVisitsBySLA"].fillna(round(df["AllowedWeeklyVisitsBySLA"].median(), 0), inplace=True)

In [159]:
df.isna().sum()

ID                            0
Age                           0
Gender                        0
Income                        0
EnrollmentStart               0
EnrollmentFinish              0
LastPeriodStart               0
LastPeriodFinish              0
DateLastVisit                 0
DaysWithoutFrequency          0
LifetimeValue                 0
UseByTime                     0
AthleticsActivities           0
WaterActivities               0
FitnessActivities             0
DanceActivities               0
TeamActivities                0
RacketActivities              0
CombatActivities              0
NatureActivities              0
SpecialActivities             0
OtherActivities               0
NumberOfFrequencies           0
AttendedClasses               0
AllowedWeeklyVisitsBySLA      0
AllowedNumberOfVisitsBySLA    0
RealNumberOfVisits            0
NumberOfRenewals              0
HasReferences                 0
NumberOfReferences            0
Dropout                       0
dtype: i